In [1]:
import os
import csv
import pandas as pd
import librosa
import numpy as np
from pathlib import Path
from tqdm import tqdm
import gc
import re
import glob
import pickle

import soundfile as sf
from musicnn.extractor import extractor as musicnn_extractor

import torch
import resampy
from transformers import Wav2Vec2Processor, Wav2Vec2Model

import warnings
warnings.filterwarnings('ignore')

c:\py\Anaconda3\envs\personal_CRM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MTGJamendoFeatureExtractor:
    def __init__(self, data_path, metadata_file="autotagging_moodtheme.tsv"):
        """
        Initialize the feature extractor for MTG-Jamendo dataset
        
        Args:
            data_path (str): Path to your data directory (e.g., "E:/Oxford/Extra/ICASSP/Draft_1/mtg-jamendo-dataset/data")
            metadata_file (str): Name of the metadata TSV file
        """
        self.data_path = Path(data_path)
        self.metadata_file = metadata_file
        self.audio_base_dir = self.data_path / "mtg-jamendo-data"
        self.metadata_df = None
        self.mapping_validation = None
        
    def load_metadata(self):
        metadata_path = self.data_path / self.metadata_file
        if not metadata_path.exists():
            raise FileNotFoundError(f"Metadata file not found at {metadata_path}")

        print(f"Loading metadata (with tag‑fix) from: {metadata_path}")

        # Read raw lines and fix rows with extra tabs in the TAGS column
        fixed_rows = []
        with open(metadata_path, 'r', encoding='utf-8', newline='') as f:
            reader = csv.reader(f, delimiter='\t')
            header = next(reader)
            num_cols = len(header)  # should be 6
            for row in reader:
                if len(row) > num_cols:
                    # join all “extra” columns into the last (TAGS) field
                    row = row[: num_cols-1 ] + ['\t'.join(row[num_cols-1:])]
                elif len(row) < num_cols:
                    # optionally skip or pad
                    row = row + ['']*(num_cols - len(row))
                fixed_rows.append(row)

        # Create DataFrame
        self.metadata_df = pd.DataFrame(fixed_rows, columns=header)
        print(f"Loaded {len(self.metadata_df)} tracks from metadata")
        print(f"Columns: {list(self.metadata_df.columns)}\n")
        print(self.metadata_df.head(3))
        return self.metadata_df
    
    def validate_audio_directory(self):
        """Validate the audio directory structure and check for numbered folders 00-99"""
        if not self.audio_base_dir.exists():
            raise FileNotFoundError(f"Audio base directory not found: {self.audio_base_dir}")
        
        # Check for numbered folders 00-99
        numbered_dirs = []
        for i in range(100):
            dir_name = f"{i:02d}"  # Format as 00, 01, 02, ..., 99
            dir_path = self.audio_base_dir / dir_name
            if dir_path.exists() and dir_path.is_dir():
                numbered_dirs.append(dir_name)
        
        print(f"Found audio base directory: {self.audio_base_dir}")
        print(f"Found {len(numbered_dirs)} numbered subdirectories (00-99)")
        
        if len(numbered_dirs) == 0:
            raise FileNotFoundError("No numbered subdirectories (00-99) found in audio directory")
        
        # Count total MP3 files
        total_mp3_count = 0
        for dir_name in numbered_dirs[:5]:  # Sample first 5 directories
            dir_path = self.audio_base_dir / dir_name
            mp3_files = list(dir_path.glob("*.mp3"))
            total_mp3_count += len(mp3_files)
            print(f"  Directory {dir_name}: {len(mp3_files)} MP3 files")
        
        print(f"Sample shows {total_mp3_count} MP3 files in first 5 directories")
        return True
    


    def get_audio_path(self, track_id):
        # assume self.metadata_df is loaded
        row = self.metadata_df.loc[self.metadata_df['TRACK_ID'] == track_id]
        if row.empty:
            raise KeyError(f"No metadata found for track {track_id}")
        relpath = row.iloc[0]['PATH']     # e.g. "48/948.mp3"
        fullpath = self.audio_base_dir / relpath
        if not fullpath.exists():
            raise FileNotFoundError(f"Audio file not found at {fullpath}")
        return fullpath
    
    def validate_mapping(self, sample_size=100):
        """
        Validate mapping between metadata TSV and actual audio files
        
        Args:
            sample_size (int): Number of random samples to check
        
        Returns:
            dict: Validation results with statistics
        """
        print(f"\n{'='*50}")
        print("VALIDATING AUDIO-METADATA MAPPING")
        print(f"{'='*50}")
        
        if self.metadata_df is None:
            self.load_metadata()
        
        # Sample tracks for validation
        sample_df = self.metadata_df.sample(min(sample_size, len(self.metadata_df)), random_state=42)
        
        validation_results = {
            'total_checked': len(sample_df),
            'found': 0,
            'missing': 0,
            'found_tracks': [],
            'missing_tracks': [],
            'sample_paths': []
        }
        
        print(f"Checking {len(sample_df)} random tracks from metadata...")
        
        # Check if we can identify the track ID column
        possible_id_columns = ['TRACK_ID', 'track_id', 'id', 'ID', 'TrackID']
        track_id_col = None
        
        for col in possible_id_columns:
            if col in self.metadata_df.columns:
                track_id_col = col
                break
        
        if track_id_col is None:
            print("Available columns:", list(self.metadata_df.columns))
            raise ValueError("Could not identify track ID column. Please check column names.")
        
        print(f"Using '{track_id_col}' as track ID column")
        
        for idx, row in sample_df.iterrows():
            track_id = row[track_id_col]
            audio_path = self.get_audio_path(track_id)
            
            if audio_path.exists():
                validation_results['found'] += 1
                validation_results['found_tracks'].append({
                    'track_id': track_id,
                    'path': str(audio_path),
                    'size_mb': audio_path.stat().st_size / (1024*1024)
                })
                # Store first 5 found paths as samples
                if len(validation_results['sample_paths']) < 5:
                    validation_results['sample_paths'].append(str(audio_path))
            else:
                validation_results['missing'] += 1
                validation_results['missing_tracks'].append({
                    'track_id': track_id,
                    'expected_path': str(audio_path)
                })
        
        # Calculate statistics
        found_percentage = (validation_results['found'] / validation_results['total_checked']) * 100
        
        print(f"\nVALIDATION RESULTS:")
        print(f"  Total checked: {validation_results['total_checked']}")
        print(f"  Found: {validation_results['found']} ({found_percentage:.1f}%)")
        print(f"  Missing: {validation_results['missing']} ({100-found_percentage:.1f}%)")
        
        if validation_results['sample_paths']:
            print(f"\nSample found audio paths:")
            for path in validation_results['sample_paths']:
                print(f"  {path}")
        
        if validation_results['missing_tracks'] and len(validation_results['missing_tracks']) <= 5:
            print(f"\nMissing tracks (showing first 5):")
            for track in validation_results['missing_tracks'][:5]:
                print(f"  Track {track['track_id']}: Expected at {track['expected_path']}")
        
        # Check folder distribution for found tracks
        if validation_results['found_tracks']:
            folder_distribution = {}
            for track in validation_results['found_tracks']:
                folder = Path(track['path']).parent.name
                folder_distribution[folder] = folder_distribution.get(folder, 0) + 1
            
            print(f"\nFolder distribution of found tracks:")
            for folder, count in sorted(folder_distribution.items()):
                print(f"  Folder {folder}: {count} tracks")
        
        self.mapping_validation = validation_results
        
        if found_percentage < 50:
            print(f"\n⚠️  WARNING: Only {found_percentage:.1f}% of tracks found!")
            print("This might indicate an issue with the folder structure or file naming.")
        else:
            print(f"\n✅ Validation successful: {found_percentage:.1f}% of tracks found")
        
        return validation_results
    
    def load_audio_file(self, file_path, sr=22050, duration=30.0):
        """
        Load an audio file using librosa
        
        Args:
            file_path (Path): Path to the audio file
            sr (int): Sample rate (default: 22050 Hz)
            duration (float): Duration to load in seconds (default: 30.0 for 30-second clips)
        
        Returns:
            tuple: (audio_data, sample_rate) or (None, None) if loading fails
        """
        try:
            # Load audio file
            y, sr_actual = librosa.load(file_path, sr=sr, duration=duration)
            return y, sr_actual
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            return None, None
    
    def process_dataset(self, feature_extractor_func, max_files=None, save_interval=100, validate_first=True):
        """
        Main processing function to extract features from all audio files
        
        Args:
            feature_extractor_func: Your feature extraction function
            max_files (int): Limit processing to first N files (for testing)
            save_interval (int): Save intermediate results every N files
            validate_first (bool): Whether to validate mapping before processing
        
        Returns:
            pd.DataFrame: DataFrame with extracted features
        """
        # Load metadata and validate audio directory
        self.load_metadata()
        self.validate_audio_directory()
        
        # Run validation check first
        if validate_first:
            validation_results = self.validate_mapping(sample_size=50)
            
            # Ask user if they want to continue if validation shows issues
            if validation_results['found'] / validation_results['total_checked'] < 0.5:
                print("\n⚠️  Low success rate in validation. Please check the issues above.")
                response = input("Do you want to continue anyway? (y/n): ")
                if response.lower() != 'y':
                    print("Processing cancelled.")
                    return None
        
        # 2) Try to resume from the latest pickle
        results = []
        processed_count = 0
        pkl_pattern = str(self.data_path / "features_intermediate_*.pkl")
        all_pkls = glob.glob(pkl_pattern)
        if all_pkls:
            # Extract numbers, find the max
            nums = [int(re.search(r'_(\d+)\.pkl$', p).group(1)) for p in all_pkls]
            latest = max(nums)
            latest_path = self.data_path / f"features_intermediate_{latest}.pkl"
            print(f"Resuming from checkpoint: {latest_path}")
            # Load it
            results = pd.read_pickle(latest_path).to_dict('records')
            processed_count = latest

        failed_count = 0
        
        # 3) Determine which rows remain to be processed
        df_full = self.metadata_df
        # If max_files is set, it caps the total to process (incl. resumed)
        total_to_do = max_files if max_files is not None else len(df_full)
        start_idx = processed_count
        df_to_process = df_full.iloc[start_idx : total_to_do]
        
        # Identify track ID column
        possible_id_columns = ['TRACK_ID', 'track_id', 'id', 'ID', 'TrackID']
        track_id_col = None
        for col in possible_id_columns:
            if col in self.metadata_df.columns:
                track_id_col = col
                break
        
        if track_id_col is None:
            raise ValueError(f"Could not identify track ID column. Available columns: {list(self.metadata_df.columns)}")
        
        print(f"Resuming processing at index {start_idx} (track #{processed_count+1})")
        print(f"Tracks left: {len(df_to_process)}")
        print(f"Using '{track_id_col}' as track ID column")
        
        # Process each track
        for idx, row in tqdm(df_to_process.iterrows(), total=len(df_to_process), desc="Processing audio"):
            track_id = row[track_id_col]
            
            # Get audio file path
            audio_path = self.get_audio_path(track_id)
            
            if not audio_path.exists():
                # print(f"Audio file not found: {audio_path}")  # Comment out to reduce noise
                failed_count += 1
                continue
            
            # Load audio
            audio_data, sample_rate = self.load_audio_file(audio_path)
            
            if audio_data is None:
                failed_count += 1
                continue
            
            # Extract features using your feature extractor
            try:
                features = feature_extractor_func(audio_data, sample_rate)
                
                # Combine metadata with features
                result_row = {
                    'track_id': track_id,
                    'audio_path': str(audio_path),
                    'duration': len(audio_data) / sample_rate,
                    'sample_rate': sample_rate,
                    **features  # Unpack feature dictionary
                }
                
                # Add original metadata columns
                for col in row.index:
                    if col not in result_row:
                        result_row[f'meta_{col}'] = row[col]
                
                results.append(result_row)
                processed_count += 1
                
                # Save intermediate results
                if processed_count % save_interval == 0:
                    temp_df = pd.DataFrame(results)
                    # 2) Save CSV checkpoint
                    csv_path = self.data_path / f"features_intermediate_{processed_count}.csv"
                    temp_df.to_csv(csv_path, index=False)  

                    # 3) Save Pickle checkpoint
                    pkl_path = self.data_path / f"features_intermediate_{processed_count}.pkl"
                    temp_df.to_pickle(pkl_path)  

                    # 4) Optional: free memory if needed
                    del temp_df  
                    gc.collect()

                    print(f"Saved intermediate results at {processed_count} files →")
                    print(f"  • CSV:  {csv_path}")
                    print(f"  • Pickle: {pkl_path}")
                
            except Exception as e:
                print(f"Feature extraction failed for {track_id}: {e}")
                failed_count += 1
        
        # Create final DataFrame
        features_df = pd.DataFrame(results)
        
        print(f"\nProcessing complete!")
        print(f"Successfully processed: {processed_count} files")
        print(f"Failed: {failed_count} files")
        if len(features_df) > 0:
            print(f"Feature DataFrame shape: {features_df.shape}")
        
        return features_df


In [3]:
def quick_validation_check():
    """
    Quick function to just validate the mapping without doing feature extraction
    Run this first to make sure everything is set up correctly
    """
    data_path = "E:/Oxford/Extra/ICASSP/Draft_1/mtg-jamendo-dataset/data"
    extractor = MTGJamendoFeatureExtractor(data_path)
    
    print("Loading metadata and validating audio directory structure...")
    extractor.load_metadata()
    extractor.validate_audio_directory()
    
    print("\nValidating audio-metadata mapping...")
    validation_results = extractor.validate_mapping(sample_size=100)
    
    return validation_results

def musicnn_feature_extractor(audio_data: np.ndarray,
                              sample_rate: int,
                              model: str = 'MTT_musicnn') -> dict:
    """
    Given raw audio (numpy) + sample rate, save to temp WAV,
    run musicnn.extractor to get intermediate representations.
    Returns a dict with keys like 'timbral', 'temporal', 'cnn1', ...,
    'penultimate' (for musicnn models) :contentReference[oaicite:5]{index=5}.
    """
    tmp_path = 'temp_clip.wav'
    sf.write(tmp_path, audio_data, sample_rate)  
    
    # extractor returns (taggram, tags, features_dict)
    _, _, feats = musicnn_extractor(tmp_path,
                             model=model,
                             extract_features=True)
    return feats


# Load once at module scope:
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')
model     = Wav2Vec2Model.from_pretrained('facebook/wav2vec2-base-960h')
model.eval()

def wav2vec_feature_extractor(audio_data: np.ndarray,
                              sample_rate: int) -> dict:
    """
    Resample to 16 kHz, tokenize with Wav2Vec2Processor,
    run Wav2Vec2Model, and average last_hidden_state to
    get one fixed-length embedding per clip :contentReference[oaicite:7]{index=7}.
    """
    # Resample if needed
    if sample_rate != 16000:
        audio_data = resampy.resample(audio_data, sample_rate, 16000)
    
    # Tokenize & run model
    inputs = processor(audio_data,
                       sampling_rate=16000,
                       return_tensors='pt',
                       padding=True)
    with torch.no_grad():
        outputs = model(**inputs)  # outputs.last_hidden_state shape (1, T, 768)
    
    # Mean‑pool over the time dimension → (768,)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return {'wav2vec2_embedding': embedding}

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
data_path = "E:/Oxford/Extra/ICASSP/Draft_1/mtg-jamendo-dataset/data"

DATA_ROOT = Path("E:/Oxford/Extra/ICASSP/Draft_1/mtg-jamendo-dataset")

In [ ]:
# 1) musicnn embeddings
extractor = MTGJamendoFeatureExtractor(data_path)
df_musicnn = extractor.process_dataset(
    feature_extractor_func=musicnn_feature_extractor,
    max_files=None,
    save_interval=1000,
    validate_first=False
)
# # display it
# print(df_musicnn.T)  # transpose so columns become rows for easier reading
df_musicnn.to_csv(DATA_ROOT / 'mtg_jamendo_musicnn_embeddings.csv',
                  index=False)
df_musicnn.to_pickle(DATA_ROOT / 'mtg_jamendo_musicnn_embeddings.pkl')

Loading metadata (with tag‑fix) from: E:\Oxford\Extra\ICASSP\Draft_1\mtg-jamendo-dataset\data\autotagging_moodtheme.tsv
Loaded 18486 tracks from metadata
Columns: ['TRACK_ID', 'ARTIST_ID', 'ALBUM_ID', 'PATH', 'DURATION', 'TAGS']

        TRACK_ID      ARTIST_ID      ALBUM_ID        PATH DURATION  \
0  track_0000948  artist_000087  album_000149  48/948.mp3    212.7   
1  track_0000950  artist_000087  album_000149  50/950.mp3    248.0   
2  track_0000951  artist_000087  album_000149  51/951.mp3    199.7   

                      TAGS  
0  mood/theme---background  
1  mood/theme---background  
2  mood/theme---background  
Found audio base directory: E:\Oxford\Extra\ICASSP\Draft_1\mtg-jamendo-dataset\data\mtg-jamendo-data
Found 100 numbered subdirectories (00-99)
  Directory 00: 202 MP3 files
  Directory 01: 200 MP3 files
  Directory 02: 207 MP3 files
  Directory 03: 190 MP3 files
  Directory 04: 193 MP3 files
Sample shows 992 MP3 files in first 5 directories
Resuming from checkpoint: E:\O

Processing audio:   0%|          | 0/12486 [00:00<?, ?it/s]

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

Processing audio:   0%|          | 1/12486 [01:30<313:21:57, 90.36s/it]

done!
Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

...

done!
Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

Processing audio:   3%|▎         | 406/12486 [15:36<6:30:35,  1.94s/it]

done!
Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

Processing audio:   3%|▎         | 407/12486 [15:39<7:13:52,  2.16s/it]

done!


In [ ]:
# 2) wav2vec2 embeddings
extractor = MTGJamendoFeatureExtractor(data_path)
df_wav2vec = extractor.process_dataset(
    feature_extractor_func=wav2vec_feature_extractor,
    max_files=None,
    save_interval=1000,
    validate_first=True
)
# # display it
# print(df_wav2vec.T)  # transpose so columns become rows for easier reading
df_wav2vec.to_csv(DATA_ROOT / 'mtg_jamendo_wav2vec2_embeddings.csv',
                  index=False)
df_wav2vec.to_pickle(DATA_ROOT / 'mtg_jamendo_wav2vec2_embeddings.pkl')

In [ ]:
emb = df_wav2vec.pop('wav2vec2_embedding').tolist()
df_wav2vec[[f'w2v_{i}' for i in range(emb[0].shape[0])]] = pd.DataFrame(emb)

df_wav2vec.to_csv(DATA_ROOT / 'mtg_jamendo_wav2vec2_embeddings_v2.csv',
                  index=False)
df_wav2vec.to_pickle(DATA_ROOT / 'mtg_jamendo_wav2vec2_embeddings_v2.pkl')